# Q_ RATE OF CHANGE

In [3]:
from koselleck import *

In [4]:
dfdecs=get_pathdf_models_bydecade()
dfdecs

,corpus,period_start,period_end,path,path_vocab,run,period,period_len
503,bpo,1700,1710,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_01,1700-1710,10
510,bpo,1700,1710,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_02,1700-1710,10
492,bpo,1700,1710,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_03,1700-1710,10
501,bpo,1700,1710,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_04,1700-1710,10
505,bpo,1700,1710,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_05,1700-1710,10
...,...,...,...,...,...,...,...,...
210,bpo,1890,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_21,1890-1900,10
219,bpo,1890,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_22,1890-1900,10
217,bpo,1890,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_23,1890-1900,10
202,bpo,1890,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_24,1890-1900,10


In [5]:
measure_change??

Signature:
measure_change(
    m1,
    m2,
    words=None,
    funcs=[<function measure_change_abstractness at 0x7fd459581e60>, <function measure_change_noiseaware at 0x7fd459581b00>, <function measure_change_procrustes at 0x7fd459581c20>, <function measure_change_local at 0x7fd459581cb0>],
    **attrs,
)
Docstring: <no docstring>
Source:   
def measure_change(
        m1,
        m2,
        words=None,
        funcs = [
            measure_change_abstractness,
            measure_change_noiseaware, 
            measure_change_procrustes,
            measure_change_local,
        ],
        **attrs):
    df=None
    for func in funcs:
        suffix=func.__name__.split('_')[-1]
        fdf=func(m1,m2,words=words,**attrs)
        if 'dist' in set(fdf.columns):
            fdf['dist']=zscore(fdf['dist'])
            fdf['rank']=(1-fdf['dist']).rank()
            fdf['perc']=fdf['dist'].apply(lambda x: percentileofscore(fdf.dist, x))
            # fdf['z']=zscore(fdf['dist'])
        fdf

In [13]:
def do_measure_change_over_decades(obj,words=None,progress=True,**y):
    try:
        row1,row2=obj
        if words is None: words=get_valid_words()
        m1,m2=load_model_row(row1),load_model_row(row2)
        odf=measure_change(m1,m2,words=words,progress=progress,
                           funcs=[measure_change_noiseaware],**y).reset_index()
        odf['period1'],odf['period2']=row1.period,row2.period
        return odf
    except Exception:
        return pd.DataFrame()

def measure_change_over_decades(dfgrp,words=None,num_proc=1,progress=True,**y):
    dfgrp=dfgrp.sort_values('period_start')
    row1=None
    m1=None
    odf=pd.DataFrame()
    #iterr=tqdm(dfgrp.iterrows(),total=len(dfgrp)) if progress else dfgrp.iterrows()
    rows=[row for i,row in dfgrp.iterrows()]
    objs=ngram(rows,2)
    resdfs=[
        x for x in pmap(
            do_measure_change_over_decades,
            objs,
            num_proc=num_proc,
            progress=progress,
            kwargs=dict(
                progress=False,
                words=words,
                **y
            )
        ) if type(x)==pd.DataFrame
    ]
    return pd.concat(resdfs) if len(resdfs) else pd.DataFrame()

In [14]:
grps=dfdecs.groupby('run')
for i,dfgrp in grps: pass

In [15]:
# gen
odf=measure_change_over_decades(dfgrp,num_proc=5)
# save
odf.to_pickle('data/data.semantic_change_over_decades.1run.pkl')
odf

Mapping do_measure_change_over_decades() [x5]: 100%|██████████| 19/19 [01:37<00:00,  5.16s/it]


,word,rank,perc,dist,rank_noiseaware,perc_noiseaware,dist_noiseaware,is_clean_noiseaware,period1,period2
0,probation,1.0,100.000000,4.346301,1.0,100.000000,4.346301,True,1700-1710,1710-1720
1,fighting,2.0,99.966375,4.128618,2.0,99.966375,4.128618,False,1700-1710,1710-1720
2,repose,3.0,99.932751,3.916678,3.0,99.932751,3.916678,False,1700-1710,1710-1720
3,inhabitant,4.0,99.899126,3.906305,4.0,99.899126,3.906305,True,1700-1710,1710-1720
4,chat,5.0,99.865501,3.886035,5.0,99.865501,3.886035,False,1700-1710,1710-1720
...,...,...,...,...,...,...,...,...,...,...
5991,character,5992.0,0.083389,-1.280767,5992.0,0.083389,-1.280767,True,1880-1890,1890-1900
5992,love,5993.0,0.066711,-1.282315,5993.0,0.066711,-1.282315,True,1880-1890,1890-1900
5993,number,5994.0,0.050033,-1.286275,5994.0,0.050033,-1.286275,True,1880-1890,1890-1900
5994,power,5995.0,0.033356,-1.307527,5995.0,0.033356,-1.307527,True,1880-1890,1890-1900


In [16]:
# gen
odf2 = pmap_groups(
    measure_change_over_decades,
    dfdecs.groupby('run'),
    num_proc=1,
    kwargs=dict(progress=False),
    desc='Measuring semantic change over all decades and runs'
).reset_index()

# save
odf2.to_pickle('data/data.semantic_change_over_decades.runs.pkl')
odf2

Measuring semantic change over the decades [x1]: 100%|██████████| 25/25 [1:22:52<00:00, 198.90s/it]


,run,word,rank,perc,dist,rank_noiseaware,perc_noiseaware,dist_noiseaware,is_clean_noiseaware,period1,period2
0,run_01,hearing,1.0,100.000000,5.626167,1.0,100.000000,5.626167,False,1700-1710,1710-1720
1,run_01,plane,2.0,99.966375,4.957808,2.0,99.966375,4.957808,True,1700-1710,1710-1720
2,run_01,dancer,3.0,99.932751,4.532773,3.0,99.932751,4.532773,True,1700-1710,1710-1720
3,run_01,mob,4.0,99.899126,4.467673,4.0,99.899126,4.467673,False,1700-1710,1710-1720
4,run_01,spaniard,5.0,99.865501,4.367539,5.0,99.865501,4.367539,True,1700-1710,1710-1720
...,...,...,...,...,...,...,...,...,...,...,...
2436995,run_25,character,5992.0,0.083389,-1.280767,5992.0,0.083389,-1.280767,True,1880-1890,1890-1900
2436996,run_25,love,5993.0,0.066711,-1.282315,5993.0,0.066711,-1.282315,True,1880-1890,1890-1900
2436997,run_25,number,5994.0,0.050033,-1.286275,5994.0,0.050033,-1.286275,True,1880-1890,1890-1900
2436998,run_25,power,5995.0,0.033356,-1.307527,5995.0,0.033356,-1.307527,True,1880-1890,1890-1900


In [17]:
odf2

,run,word,rank,perc,dist,rank_noiseaware,perc_noiseaware,dist_noiseaware,is_clean_noiseaware,period1,period2
0,run_01,hearing,1.0,100.000000,5.626167,1.0,100.000000,5.626167,False,1700-1710,1710-1720
1,run_01,plane,2.0,99.966375,4.957808,2.0,99.966375,4.957808,True,1700-1710,1710-1720
2,run_01,dancer,3.0,99.932751,4.532773,3.0,99.932751,4.532773,True,1700-1710,1710-1720
3,run_01,mob,4.0,99.899126,4.467673,4.0,99.899126,4.467673,False,1700-1710,1710-1720
4,run_01,spaniard,5.0,99.865501,4.367539,5.0,99.865501,4.367539,True,1700-1710,1710-1720
...,...,...,...,...,...,...,...,...,...,...,...
2436995,run_25,character,5992.0,0.083389,-1.280767,5992.0,0.083389,-1.280767,True,1880-1890,1890-1900
2436996,run_25,love,5993.0,0.066711,-1.282315,5993.0,0.066711,-1.282315,True,1880-1890,1890-1900
2436997,run_25,number,5994.0,0.050033,-1.286275,5994.0,0.050033,-1.286275,True,1880-1890,1890-1900
2436998,run_25,power,5995.0,0.033356,-1.307527,5995.0,0.033356,-1.307527,True,1880-1890,1890-1900
